In [58]:
import os
import re
from nltk.tokenize import TreebankWordTokenizer
import pandas as pd

from nltk.stem import WordNetLemmatizer

In [59]:
ready_df = pd.DataFrame()

In [60]:
def preprocess(text):

    tokenized_text = TreebankWordTokenizer().tokenize(text)
    
    wordsFiltered = []

    for w in tokenized_text:
        for wf in re.findall(r'(?u)(\w+)', w):
            #for wfreal in re.findall(r'(\D+)', wf):
            wordsFiltered.append(''.join(wf))
    
    return wordsFiltered

In [61]:
def get_context(file_path):
    
    with open(file_path) as fl:
        try:
            text = fl.read()
        except:
            print ("Error: " + file_path )
    
    reg_term = re.compile(r'&(.*?)!&')
    indexes = []
    for i in reg_term.finditer(text):
        indexes.append((i.start(), i.end()))

    left_context = []
    right_context = []
    left_word = []

    for ind in indexes:
        left_part = text[:ind[0]]
        right_part = text[ind[1]+1:]
        clear_left = preprocess(left_part)
        clear_right = preprocess(right_part)
    
    
        #left context
        if len(clear_left) > 5:
            left_context.append(clear_left[-5:])
            left_word.append(clear_left[-1])
        else:
            left_context.append(clear_left)
            if len(clear_left) > 0:
                left_word.append(clear_left[-1])
            else:
                left_word.append(' ')
    
        #right context
        if len(clear_right) > 5:
            right_context.append(clear_right[:5])
        else:
            right_context.append(clear_right)
            
    
    terms = reg_term.findall(text)
    
    global ready_df
    lemmatizer = WordNetLemmatizer()
    
    
    for i in range(len(terms)):
        lft = left_context[i]
        lw = str(left_word[i])
        term = terms[i]
        rgt = right_context[i]
        lc = ' '.join(lft)
        rc = ' '.join(rgt)
        
        df = pd.DataFrame([[lc, lw, term, rc]],
                      columns=['left_context', 'left_word', 'term', 'right_context'])
        ready_df = ready_df.append(df)


In [62]:
start_path = './terms_markup/Marked/'

for path, dirs, files in os.walk(start_path):

    for fname in files:
        if not fname.startswith('.'):
            get_context(start_path+fname)


In [63]:
len(ready_df)

1133

In [64]:
#ready_df.tail(500)

In [65]:
df_sorted = ready_df.sort_values(['left_word'])

In [66]:
df_sorted.head()

,left_context,left_word,term,right_context
0,,,Маглев,на тестовом запуске в префектуре
0,,,Облако,с точки зрения пользователя это
0,,,Изменение возмущений в грунте,в зависимости от скорости снаряда
0,уроки соседа сверху каждые 10,10,децибел,громкость звука становится в два
0,в нашей на расстоянии 1350,1350,световых лет,от Земли в созвездии Ориона


In [67]:
ready_df.to_csv('terms_context.csv', sep='\t')

In [68]:
df_sorted.to_csv('terms_sorted.tsv', sep = '\t')

In [69]:
lemma = WordNetLemmatizer()

In [70]:
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tag import tnt
from nltk.corpus import treebank
from nltk.corpus.reader.conll import *

In [ ]:
train_file2 = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu'

In [71]:
train_file = 'unamb_sent_14_6.conllu'
train_reader = ConllCorpusReader(root = '', fileids = [train_file], columntypes = ['ignore', 'words', 'ignore', 'pos', 'chunk'])


sents = list(train_reader.iob_sents())
train_sents = sents[:30000]
test_sents = sents[30000:]

print(len(train_sents))
print(len(test_sents))

30000
8508


In [33]:
test_sents_1 = []
for sent in test_sents:
    sent_1 = []
    for word in sent:
        sent_1.append(word[:2])
    test_sents_1.append(sent_1)
print(test_sents_1[0])

train_sents_1 = []
for sent in test_sents:
    sent_1 = []
    for word in sent:
        sent_1.append(word[:2])
    train_sents_1.append(sent_1)
print(test_sents_1[0])

[('Письмо', 'NOUN'), ('восьмое', 'ADJ'), ('посвящено', 'ADJ'), ('правилам', 'NOUN'), (',', 'PUNCT'), ('благоприятным', 'ADJ'), ('для', 'ADP'), ('всевозможных', 'ADJ'), ('грешников', 'NOUN'), (':', 'PUNCT'), ('воров', 'NOUN'), (',', 'PUNCT'), ('банкротов', 'NOUN'), (',', 'PUNCT'), ('нечестных', 'ADJ'), ('судей', 'NOUN'), (',', 'PUNCT'), ('распутных', 'ADJ'), ('женщин', 'NOUN'), ('.', 'PUNCT')]
[('Письмо', 'NOUN'), ('восьмое', 'ADJ'), ('посвящено', 'ADJ'), ('правилам', 'NOUN'), (',', 'PUNCT'), ('благоприятным', 'ADJ'), ('для', 'ADP'), ('всевозможных', 'ADJ'), ('грешников', 'NOUN'), (':', 'PUNCT'), ('воров', 'NOUN'), (',', 'PUNCT'), ('банкротов', 'NOUN'), (',', 'PUNCT'), ('нечестных', 'ADJ'), ('судей', 'NOUN'), (',', 'PUNCT'), ('распутных', 'ADJ'), ('женщин', 'NOUN'), ('.', 'PUNCT')]


In [34]:
tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_sents_1)
tnt_pos_tagger.evaluate(test_sents_1)

0.9953090882497362

In [44]:
from tqdm import tqdm

patterns = []
for sent in tqdm(df_sorted['left_context']):
    try:
        POS = m.analyze(sent.lower())
        patterns.append(POS)
    except:
        continue

    

100%|██████████| 1133/1133 [00:02<00:00, 482.58it/s]


In [46]:
patterns[:5]

[[],
 [],
 [],
 [{'analysis': [{'gr': 'S,муж,неод=(вин,мн|им,мн)', 'lex': 'урок'}],
   'text': 'уроки'},
  {'text': ' '},
  {'analysis': [{'gr': 'S,муж,од=(вин,ед|род,ед)', 'lex': 'сосед'}],
   'text': 'соседа'},
  {'text': ' '},
  {'analysis': [{'gr': 'ADV=', 'lex': 'сверху'}], 'text': 'сверху'},
  {'text': ' '},
  {'analysis': [{'gr': 'APRO=(им,мн|вин,мн,неод)', 'lex': 'каждый'}],
   'text': 'каждые'},
  {'text': ' '},
  {'text': '10'},
  {'text': '\n'}],
 [{'analysis': [{'gr': 'PR=', 'lex': 'в'}], 'text': 'в'},
  {'text': ' '},
  {'analysis': [{'gr': 'APRO=(пр,ед,жен|дат,ед,жен|род,ед,жен|твор,ед,жен)',
     'lex': 'наш'}],
   'text': 'нашей'},
  {'text': ' '},
  {'analysis': [{'gr': 'PR=', 'lex': 'на'}], 'text': 'на'},
  {'text': ' '},
  {'analysis': [{'gr': 'S,сред,неод=пр,ед', 'lex': 'расстояние'}],
   'text': 'расстоянии'},
  {'text': ' '},
  {'text': '1350'},
  {'text': '\n'}]]

In [43]:
from pymystem3 import Mystem

m = Mystem()

In [48]:
unigrams = []
bigrams = []
trigrams = []
multigrams = []

for term in df_sorted['term']:
    if len(term.split()) == 1:
        unigrams.append(term)
    elif len(term.split()) == 2:
        bigrams.append(term)
    elif len(term.split()) == 3:
        trigrams.append(term)
    else:
        multigrams.append(term)
        

In [49]:
len(unigrams)

551

In [50]:
len(bigrams)

464

In [51]:
len(trigrams)

77

In [52]:
len(multigrams)

41

In [53]:
print(bigrams)

['световых лет', 'ground-penetrating radar', 'проводящие нити', 'магнитная левитация', 'митохондриального генома', 'комплексном мире', 'гравитационных волн', 'интериоризированный кодекс', 'оксид церия', 'Внешний суверенитет', 'квантовая физика', 'угольные микрофоны', 'клеток лейкемии', 'гидрида кремния', 'проводящих нитей', 'демонологические идеи', 'внеземных геномов', 'Излучение Хокинга', 'фактор Xa', 'фибрилляции предсердий', 'нейрологические субстраты', 'Иммунный ответ', 'Радиоуглеродное датирование', 'посадочного модуля', 'гравитационное излучение', 'Черные дыры', 'египтологических исследований', 'египтологических исследований', 'регулятивные механизмы', 'нитчатый белок', 'плутониевый электрогенератор', 'бихевиористского направления', 'системы вознаграждения', 'гравитационных волн', 'скалярных возмущений', 'радиоактивных веществ', 'Асимметричное шифрование', 'дозу никотина', 'утилизировать вещества', 'интегральный показатель', 'силиконовое полотно', 'дыхательные пути', 'электрическ

In [54]:
print(unigrams)

['Маглев', 'Облако', 'децибел', 'нанометра', 'нанометра', 'наночастицы', 'дрозофилы', 'иммунолог', 'DDoS-атака', 'катализатора', 'гаджеты', 'функция', 'кривая', 'катализатора', 'зонда', 'Гальванометр', 'эксгумировали', 'катушке', 'никотина', 'лейкемия', 'либерализации', 'ривароксабан', 'Самцы', 'дендроклиматологии', 'портал', 'биомассы', 'геном', 'билингв', 'волокна', 'наночастицы', 'психофармакологии', 'грозоотметчик', 'storytelling', 'Пропофол', 'анестезиология', 'катализаторы', 'Маржинальность', 'тромбин', 'углерода', 'ривароксабана', 'самоидентификация', 'Наукометрия', 'девайсы', 'углерод', 'фрустрации', 'ТУИХ', 'волокна', 'особи', 'ТУИХ', 'интенсифицируется', 'нейробиолог', 'NH4+', 'Анестезиологи', 'аэронавтика', 'DNS-провайдер', 'дрозофил', 'варфарина', 'Билингвы', 'славянофилов', 'экспоненциально', 'излучение', 'отсеквенирован', 'репозитории', 'гипоталамусе', 'облаке', 'биомедицине', 'IP-адреса', 'метаисследовании', 'тромбин', 'метан', 'аммоний', 'HDTV-качестве', 'иммунотерапии'

In [55]:
print(trigrams)

['Динамика цветовых изменений', 'Система внутреннего подкрепления', 'Комплексные дифференциальные уравнения', 'система внутреннего подкрепления', 'систему внутреннего подкрепления', 'плутониевому источнику энергии', 'руководителя по продажам', 'плотные слои атмосферы', 'технике листового золочения', 'плотных слоях атмосферы', 'пресноводном резервуарном эффекте', 'географического способа избавления', 'реакцию иммунной системы', 'вторая волна миграции', 'коре больших полушарий', 'интериоризированному моральному кодексу', 'функциональной магнитно-резонансной томографии', 'простых больших чисел', 'острая лимфобластная лейкемия', 'неклапанной фибрилляции предсердий', 'идеология русского европеизма', 'электрическую активность испытуемых', 'сопротивление в цепи', 'доминирования наукометрического подхода', 'функциональной магнитно-резонансной томографии', 'модифицированных донорских Т-клеток', 'первой генетической редакции', 'система внутреннего подкрепления', 'Ишемическая болезнь сердца', 'ма

In [56]:
print(multigrams)

['Изменение возмущений в грунте', 'активированная форма фактора десять', 'передняя часть островковой коры', 'ближнем инфракрасном, видимом и ультрафиолетовом диапазоне', 'общем разрезе профилей опросов', 'СН4&!. Таким образом, учёные продемонстрировали, что физические изменения &катализатора', 'антикоагулянтных (против свертывания крови) препаратов', 'Наноструктурирование металлов и сплавов', 'узлов с большим весом', 'носители языков полинезийской группы', 'носители языков полинезийской группы', 'применены в массовом порядке', 'передние области височных долей', 'оксидов титана и ниобия', 'широким окном округления алгоритма', 'поверхности внутренней выстилки сосудов', 'периоду позднего, но не среднего плейстоцена', 'Двойной слепой рандомизированный плацебо-контролируемый метод', 'независимая и зависимая переменные', 'Амплитуды их вызванных потенциалов', 'исполнительным и эмоциональным центрами головного мозга', 'возбуждении центральной нервной системы', 'популяций специфических иммунных

In [74]:
with open('rus_bigram.txt') as fl:
    bi = fl.read()
    
with open('rus_trigram.txt') as fl:
    tri = fl.read()
    

In [79]:
bi_freq = bi.split('\n')
tri_freq = tri.split('\n')

In [87]:
def text_to_dict(list_of_ngrams):
        
    unique_ngrams = list(set(list_of_ngrams))

    numbers = []
    for item in unique_ngrams:
        word_num = list_of_ngrams.count(item)
        numbers.append(word_num)
    
    dictionary = dict(zip(unique_ngrams, numbers))
    
    return dictionary

In [89]:
def get_ngrams(list_of_words, n):
    ngrams = []
    for word in list_of_words:
        for i in range(len(word)-n+1):
            ngrams.append(word[i:i+n].lower())

    n_dict = text_to_dict(ngrams)
    
    return n_dict

In [95]:
terms = []
for term in df_sorted['term']:
    terms.append(term.lower().split())

In [101]:
flat_list = [item for sublist in terms for item in sublist]

In [103]:
term_bi = get_ngrams(flat_list, 2)

In [104]:
print(term_bi)

{'нт': 85, 'ns': 3, 'yc': 2, 'ca': 1, 'ыб': 1, 'иж': 1, 'сг': 2, 'рш': 6, 'ио': 96, 'гр': 42, 'ьд': 1, 'хп': 2, '-к': 13, 'ях': 1, 'фл': 6, 'ей': 63, 'ел': 60, 'na': 1, 'шн': 10, 'ум': 12, 'ae': 2, 'te': 6, 'рф': 15, 'es': 4, 'ош': 2, 'ек': 88, 'из': 104, 'll': 2, 'фо': 28, 'ad': 1, 'то': 172, '&!': 2, 'ян': 11, 'vi': 3, 'иц': 29, 'ry': 1, 'оф': 30, 'an': 1, 'юд': 2, 'уз': 5, 'на': 194, 'ра': 202, 'ar': 4, 'ha': 2, 'ых': 75, 'дя': 3, 'бк': 1, 'au': 1, 'не': 120, 'ай': 12, 'ba': 2, 'xa': 3, 'аф': 14, 'йн': 9, 'бс': 6, 'ён': 1, 'ва': 127, 'po': 2, 'rr': 1, 'дч': 2, 'ne': 2, 'ev': 3, 'ту': 40, '-т': 2, 'о-': 10, 'ой': 111, 'ин': 153, 'т-': 9, 'дк': 12, 'ау': 12, 'пы': 12, 'ая': 44, 'ci': 1, '-ф': 2, 'ie': 1, 'ан': 198, 'et': 3, 'зц': 2, 'кр': 40, 'o2': 4, 'др': 17, 'my': 2, 'вя': 5, 'ро': 262, 'se': 1, 'в-': 1, 'mi': 1, 'бу': 6, 'ri': 1, 'зв': 12, 'нз': 2, 'yt': 2, 'кц': 25, 'ад': 32, 'с3': 1, 'дж': 8, '6n': 1, 'eo': 1, 'ъю': 1, 'ль': 107, 'мс': 3, 'рб': 3, 'дл': 5, 'тю': 9, 'сh': 1, 'их'

In [109]:
from collections import Counter

top_bi = []
for i in range(300):
    top_bi.append(Counter(term_bi).most_common(300)[i][0])

In [111]:
print(top_bi)

['но', 'ро', 'ен', 'ст', 'ни', 'ер', 'ра', 'ан', 'на', 'ко', 'ле', 'ов', 'то', 'ны', 'ал', 'те', 'ци', 'ре', 'ин', 'ти', 'он', 'ом', 'ор', 'ог', 'ли', 'ри', 'ол', 'ва', 'ия', 'тр', 'та', 'не', 'ет', 'ло', 'ой', 'од', 'ат', 'ос', 'ма', 'ль', 'нн', 'из', 'ес', 'ск', 'по', 'го', 'ок', 'че', 'ио', 'ми', 'ка', 'во', 'от', 'ве', 'ки', 'ме', 'ар', 'де', 'пр', 'ии', 'ек', 'ем', 'ит', 'си', 'нт', 'ис', 'кт', 'ак', 'ди', 'ых', 'мо', 'ьн', 'за', 'ац', 'кс', 'фи', 'ич', 'ам', 'ей', 'ик', 'ие', 'ги', 'ел', 'ир', 'да', 'пл', 'ив', 'ые', 'рн', 'ид', 'ед', 'ас', 'ий', 'вн', 'ру', 'оз', 'ла', 'им', 'ил', 'ге', 'зо', 'оп', 'ев', 'до', 'ая', 'зи', 'об', 'гр', 'би', 'бо', 'са', 'гл', 'нд', 'тн', 'ту', 'кр', 'се', 'тв', 'ез', 'иа', 'аз', 'ви', 'дн', 'со', 'фа', 'их', 'га', 'ул', 'ля', 'кл', 'ну', 'му', 'чн', 'ды', 'еп', 'ое', 'ры', 'пе', 'ав', 'аг', 'ад', 'нк', 'ег', 'зм', 'уг', 'ты', 'ку', 'па', 'оф', 'сп', 'ть', 'вы', 'ще', 'иц', 'сл', 'рт', 'ап', 'ым', 'фо', 'ый', 'фр', 'ию', 'ун', 'ке', 'иб', 'бр', 'оч

In [112]:
print(bi_freq)

['ст', 'ра', 'ни', 'во', 'пр', 'но', 'ов', 'ен', 'на', 'об', 'ть', 'ав', 'ве', 'ос', 'ва', 'го', 'тв', 'ат', 'од', 'то', 'ел', 'ли', 'че', 'им', 'ес', 'ан', 'ка', 'ны', 'ре', 'ол', 'та', 'ло', 'ль', 'ет', 'по', 'ко', 'жд', 'де', 'не', 'со', 'ме', 'ро', 'бо', 'ог', 'до', 'ия', 'ри', 'ек', 'ак', 'ие', 'св', 'ци', 'ин', 'аж', 'ор', 'ти', 'те', 'ле', 'за', 'от', 'ер', 'ый', 'ит', 'ще', 'ем', 'ды', 'ьн', 'ед', 'ди', 'ми', 'ом', 'ся', 'ар', 'же', 'ил', 'нн', 'ее', 'бе', 'ла', 'ых', 'ег', 'да', 'ья', 'мо', 'вн', 'ас', 'ац', 'ей', 'бы', 'ой', 'ал', 'ии', 'ту', 'ам', 'се', 'ич', 'вл', 'он', 'ьс', 'ое', 'вс', 'ию', 'их', 'из', 'су', 'ви', 'бр', 'уд', 'ля', 'ым', 'тр', 'ду', 'ий', 'сп', 'ру', 'ма', 'ис', 'аз', 'ск', 'дн', 'лж', 'ше', 'ож', 'ив', 'бщ', 'еж', 'уп', 'сн', 'еч', 'щи', 'жн', 'ча', 'пе', 'вы', 'сл', 'ыт', 'ез', 'лю', 'дл', 'му', 'га', 'зн', 'кл', 'ои', 'гр', 'зо', 'чт', 'оз', 'кт', 'жи', 'чн', 'ео', 'ну', 'ая', 'сс', 'ио', 'ую', 'чи', 'ги', 'оя', 'уч', 'тн', 'ик', 'ащ', 'нс', 'тс', 'ущ

In [117]:
unique_bi = []
for i in top_bi:
    if i not in bi_freq:
        unique_bi.append(i)

In [118]:
print(unique_bi)

['кс', 'фи', 'ге', 'ев', 'зи', 'нд', 'фа', 'нк', 'рт', 'фо', 'фр', 'ке', 'оч', 'кц', 'ба', 'ук', 'ха', 'мн', 'рм', 'иф', 'лл', 'ыр', 'мп', 'нг', 'ещ', 'мм', 'рф', 'лы', 'пп', 'зе', 'рк', 'уа', 'ип', 'аф', 'еф', 'эл', 'фу', '-к', 'рв', 'пи', 'гу', 'йк', 'ум', 'ау', 'пы', 'уе', 'эм', 'оо', 'нф', 'фф', 'яц', 'йр', 'хн', 'шн', 'о-', 'т-', 'тю', 'юр', 'er', 'сф', 'дж', 'рд', 'тк', 'хи', 'уш', 'эф', 'co', 'пс', 'ык', 'шк', 'om', 'эк', 'ыв']


In [113]:
term_tri = get_ngrams(flat_list, 3)

top_tri = []
for i in range(300):
    top_tri.append(Counter(term_tri).most_common(300)[i][0])

In [115]:
print(top_tri)

['ени', 'льн', 'ого', 'ния', 'аци', 'аль', 'ост', 'ной', 'ных', 'тал', 'оло', 'ров', 'тор', 'про', 'ног', 'ова', 'али', 'еск', 'ван', 'чес', 'иче', 'ные', 'лог', 'нос', 'ани', 'тро', 'сти', 'нны', 'ско', 'ист', 'енн', 'сте', 'ион', 'лиз', 'ент', 'род', 'нно', 'цио', 'ном', 'акт', 'оги', 'иза', 'ски', 'рен', 'ств', 'лек', 'ато', 'ест', 'ите', 'нит', 'иро', 'ект', 'ори', 'ерн', 'ции', 'ьно', 'ная', 'ком', 'окс', 'ние', 'тра', 'ано', 'ром', 'вер', 'стр', 'ген', 'ере', 'гле', 'она', 'лен', 'ами', 'ето', 'кат', 'нал', 'мет', 'тов', 'тер', 'анн', 'ина', 'нан', 'ден', 'тел', 'пле', 'ото', 'ата', 'кси', 'угл', 'еро', 'зат', 'екс', 'чно', 'сис', 'рин', 'гра', 'кци', 'ьны', 'рак', 'аст', 'онн', 'кон', 'тич', 'сть', 'лин', 'ома', 'тем', 'ным', 'лет', 'ери', 'рны', 'пол', 'вар', 'ово', 'лов', 'ера', 'оро', 'оно', 'ива', 'ико', 'ный', 'иал', 'ник', 'рем', 'ний', 'тив', 'оле', 'лей', 'ана', 'ора', 'изм', 'ция', 'нти', 'ред', 'кой', 'ких', 'чер', 'кле', 'рно', 'оти', 'ези', 'тно', 'ктр', 'лер', 'нов'

In [116]:
print(tri_freq)

['рав', 'ств', 'пра', 'ени', 'ать', 'ове', 'ани', 'лов', 'сво', 'ело', 'чел', 'ост', 'ого', 'ния', 'ста', 'ест', 'век', 'аво', 'ажд', 'ние', 'льн', 'име', 'ова', 'при', 'каж', 'обо', 'или', 'жды', 'тат', 'воб', 'дый', 'бод', 'аци', 'его', 'еет', 'тья', 'мее', 'лен', 'тва', 'ных', 'енн', 'тво', 'аль', 'ван', 'сто', 'ьно', 'про', 'ног', 'ако', 'нов', 'сти', 'бра', 'пол', 'олж', 'дол', 'все', 'раз', 'ным', 'тор', 'нос', 'жен', 'ред', 'общ', 'ель', 'тел', 'оди', 'чес', 'как', 'сно', 'тве', 'има', 'пре', 'щес', 'ним', 'вле', 'нны', 'тьс', 'нно', 'шен', 'ься', 'аст', 'осу', 'быт', 'сов', 'ыть', 'осн', 'одн', 'лжн', 'нию', 'ран', 'бще', 'что', 'ват', 'оль', 'чен', 'сту', 'иче', 'рес', 'ном', 'ден', 'ежд', 'нац', 'вер', 'оже', 'рин', 'вен', 'туп', 'род', 'зов', 'нар', 'стр', 'обр', 'еоб', 'дос', 'есп', 'она', 'ото', 'ьны', 'лич', 'нии', 'тоя', 'аро', 'бес', 'гос', 'кот', 'суд', 'жно', 'ное', 'ава', 'под', 'печ', 'нст', 'уда', 'дно', 'ной', 'авн', 'ече', 'ход', 'нен', 'ции', 'уще', 'ате', 'льс'

In [119]:
unique_tri = []
for i in top_tri:
    if i not in tri_freq:
        unique_tri.append(i)

In [120]:
print(unique_tri)

['тал', 'оло', 'али', 'ные', 'лог', 'тро', 'ист', 'сте', 'лиз', 'ент', 'акт', 'оги', 'иза', 'ски', 'рен', 'лек', 'ато', 'нит', 'иро', 'ект', 'ори', 'ерн', 'ная', 'ком', 'окс', 'ген', 'гле', 'ето', 'кат', 'мет', 'тов', 'нан', 'ата', 'кси', 'угл', 'еро', 'зат', 'екс', 'сис', 'кци', 'онн', 'тич', 'лин', 'ома', 'тем', 'лет', 'ери', 'рны', 'вар', 'ера', 'оно', 'ива', 'ико', 'ный', 'тив', 'оле', 'лей', 'ана', 'ора', 'изм', 'ция', 'нти', 'ких', 'чер', 'кле', 'рно', 'оти', 'ези', 'тно', 'ктр', 'лер', 'ивн', 'дыр', 'рон', 'био', 'мен', 'нде', 'зон', 'офи', 'тур', 'лев', 'иде', 'реп', 'фак', 'ней', 'ино', 'оме', 'диа', 'ток', 'бан', 'кол', 'маг', 'еле', 'цию', 'ант', 'омб', 'фра', 'емн', 'нер', 'ади', 'уве', 'оки', 'еще', 'кса', 'фар', 'сид', 'ици', 'рок', 'вой', 'кта', 'оли', 'емы', 'епо', 'тет', 'фил', 'рот', 'сув', 'рив', 'уля', 'вод', 'тез', 'опл', 'рад', 'ибр', 'тин', 'кул', 'овы', 'ифи', 'ков', 'уче', 'нол', 'гни', 'топ', 'мат', 'бри', 'ида', 'етр', 'фун', 'мед', 'ане', 'пер', 'ита', 'агн'

In [121]:
term_4 = get_ngrams(flat_list, 4)

top_4 = []
for i in range(300):
    top_4.append(Counter(term_4).most_common(300)[i][0])

In [122]:
unique_4 = []
for i in top_tri:
    if i not in tri_freq:
        unique_tri.append(i)

In [124]:
print(top_4)

['альн', 'ческ', 'ичес', 'ного', 'ения', 'олог', 'цион', 'ност', 'ализ', 'логи', 'лиза', 'ован', 'рова', 'иров', 'льно', 'атор', 'угле', 'вани', 'ески', 'сист', 'льны', 'нных', 'стем', 'тали', 'исте', 'нано', 'ката', 'зато', 'ости', 'изат', 'атал', 'окси', 'ение', 'ания', 'лени', 'ектр', 'еств', 'тром', 'ните', 'ерод', 'леро', 'глер', 'еско', 'ерен', 'ации', 'черн', 'тора', 'онал', 'иона', 'ионн', 'ацио', 'енно', 'ость', 'наль', 'чени', 'факт', 'ской', 'клет', 'вере', 'ител', 'щест', 'лект', 'енит', 'ромб', 'ьной', 'прот', 'ских', 'нной', 'фрак', 'ракт', 'ксид', 'рени', 'ация', 'асти', 'ремн', 'анны', 'ктро', 'тивн', 'суве', 'увер', 'ично', 'енти', 'иден', 'дент', 'енны', 'учен', 'меди', 'лето', 'веще', 'ради', 'ванн', 'тичн', 'рива', 'элек', 'акто', 'итет', 'ибри', 'ноло', 'чнос', 'ещес', 'тель', 'част', 'иаци', 'метр', 'нные', 'стру', 'фибр', 'гнит', 'тери', 'акта', 'нико', 'едиа', 'нкци', 'диац', 'ктор', 'магн', 'таци', 'лейк', 'агни', 'селе', 'ства', 'лекс', 'ункц', 'гиче', 'граф',

In [129]:
fl = open('4grams.txt', 'w')
for t in top_4:
    fl.write(t + '\n')

fl.close()

In [130]:
fl = open('trigrams.txt', 'w')
for t in unique_tri:
    fl.write(t + '\n')

fl.close()

In [131]:
fl = open('bigrams.txt', 'w')
for t in unique_bi:
    fl.write(t + '\n')

fl.close()